In [15]:
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from scipy import stats
from sklearn.metrics import mean_squared_error
import numpy as np
from copy import deepcopy
from numpy import inf
from math import exp
from datetime import timedelta
from sklearn.metrics import r2_score
from scipy.special import softmax
import warnings
import os
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from plotly.graph_objects import Layout
from datetime import datetime, date

warnings.simplefilter("ignore")

try:
    os.makedirs('plots')
except OSError as e:
	pass



In [ ]:
filename = 'Jan-April-20.csv'

df = pd.read_csv(filename)
df['date'] = pd.to_datetime(df.Date)

states = list(pd.unique(df['state_residence']))

In [16]:
df['state_residence'].unique()

array(['ANDAMAN AND NICOBAR ISLANDS', 'ANDHRA PRADESH',
       'ARUNACHAL PRADESH', 'ASSAM', 'BIHAR', 'CHANDIGARH',
       'CHHATTISGARH', 'DADRA AND NAGAR HAVELI', 'DAMAN AND DIU', 'DELHI',
       'GOA', 'GUJARAT', 'HARYANA', 'HIMACHAL PRADESH', 'INDIA',
       'JAMMU AND KASHMIR', 'JHARKHAND', 'KARNATAKA', 'KERALA', 'LADAKH',
       'LAKSHADWEEP', 'MADHYA PRADESH', 'MAHARASHTRA', 'MANIPUR',
       'MEGHALAYA', 'MIZORAM', 'NAGALAND', 'ODISHA', 'PUDUCHERRY',
       'PUNJAB', 'RAJASTHAN', 'SIKKIM', 'TAMIL NADU', 'TELANGANA',
       'TRIPURA', 'UTTAR PRADESH', 'UTTARAKHAND', 'WEST BENGAL'],
      dtype=object)

In [17]:
def weib(x, k, a, b, g):
	return k * g * b * (a ** b) * np.exp(-1 * g * ((a / x)  ** b)) / (x ** (b + 1))

def weib2(x, k, a, b, g, k2, a2):
	return weib(x, k, a, b, g) + weib(x, k2, a2, b, g)

def weib3(x, k, a, b, g, k2, a2, k3, a3):
	return weib(x, k, a, b, g) + weib(x, k2, a2, b, g) + weib(x, k3, a3, b, g)

def weib4(x, k, a, b, g, k2, a2, k3, a3, k4, a4):
	return weib(x, k, a, b, g) + weib(x, k2, a2, b, g) + weib(x, k3, a3, b, g) + weib(x, k4, a4, b, g)


In [ ]:
def getInfostate(df2, isdead):
	df2['Delta'] = (df2.date - min(df2.date)).dt.days
	startDate = min(df2.date)
	totalLength = max(df2.Delta)
	confirmed = []; new = []
	df2 = df2.fillna(0)
	for day in range(totalLength):
		newc = max(0, int(sum(df2.new_case[df2.Delta == day] if not isdead else df2.new_deaths[df2.Delta == day])))
		new.append(newc)
		confirmed.append(new[-1] + (confirmed[-1] if len(confirmed) > 1 else 0))
	return startDate, totalLength, confirmed, new

def totalExpected(func, popt, data):
	total = 0; day = 1
	while True:
		today = func(day, *popt) if day >= len(data) else data[day]
		total += today
		day += 1
		if day > data.index(max(data)) and today <= 1: break
	return day, total

def calcWhen(func, popt, match, data):
	total = 0; day = 1
	while True:
		today = func(day, *popt) if day >= len(data) else data[day]
		total += today
		day += 1
		if total >= match or (today == 0 and day > data.index(max(data))): break
	return day

def iterativeCurveFit(func, x, y, start):
	outliersweight = None
	for i in range(10):
		popt, pcov = curve_fit(func, x, y, start, sigma=outliersweight, maxfev=100000)
		pred = np.array([func(px, *popt) for px in x])
		old = outliersweight
		outliersweight = np.abs(pred - y)
		outliersweight = outliersweight - np.tanh(outliersweight)
		outliersweight = outliersweight / np.max(outliersweight)
		outliersweight = softmax(1 - outliersweight)
		if i > 1 and sum(abs(old - outliersweight)) < 0.001: break
	return popt, pcov

def seriesIterativeCurveFit(func, xIn, yIn, start):
	res = []; errors = []
	for ignore in range(5, 0, -1):
		x = xIn[:-1*ignore]; y = yIn[:-1*ignore]
		outliersweight = None
		for i in range(10):
			try:
				popt, pcov = curve_fit(func, x, y, start, sigma=outliersweight, absolute_sigma=True, maxfev=1000000)
			except Exception as e: 
				print('ignore -', ignore, ', exception -', e)
				break
			pred = np.array([func(px, *popt) for px in x])
			old = outliersweight
			outliersweight = np.abs(pred - y)
			outliersweight = outliersweight / np.max(outliersweight)
			outliersweight = outliersweight - np.tanh(outliersweight)
			outliersweight = softmax(1 - outliersweight)
			if i > 1 and sum(abs(old - outliersweight)) < 0.001: break
		pred = [func(px, *popt) for px in xIn]
		res.append((popt, pcov, ignore))
		errors.append(mean_absolute_percentage_error(yIn, pred))
	print(errors)
	val = res[errors.index(min(errors))]
	return val[0], val[1]

In [ ]:
def getMaxCases(y, data):
	m = 0; dday = 0
	for day,cases in enumerate(y):
		if day < len(data):
			if data[day] > m:
				m = data[day]; dday = day
		else:
			if cases > m:
				m = cases; dday = day
	return m, dday

def mean_absolute_percentage_error(y_true, y_pred): 
	return np.mean(np.abs((np.array(y_true) - np.array(y_pred)) / (np.array(y_true)+1))) * 100

def getcummulative(l):
	res = []; s = 0
	for i in l:
		s += i; res.append(s)
	return res

In [19]:
dead = False
finaldata = []
dfPlot = pd.DataFrame()
dfcPlot = pd.DataFrame()
training_data = -1
interactive = ['DELHI', 'HARYANA', 'PUNJAB', 'BIHAR']
num_peaks = dict(zip(interactive, [2, 2, 2, 2]))
for state in interactive:
	try:
		dead = False
		print("Predicting For ", state)
		df2 = df[df['state_residence'] == state]
		res = getInfostate(df2, False)
		data = res[-1]
		days = res[1]
		start = res[0]

		func = [(weib, [60000, 14, 4, 500], [2000, 54, 4, 500]), 
				(weib2, [60000, 14, 4, 500, 60000, 14], [2000, 54, 4, 500, 2000, 54]), 
				(weib3, [60000, 14, 4, 500, 60000, 14, 60000, 14], [2000, 54, 4, 500, 2000, 54, 2000, 54]),
				(weib4, [60000, 14, 4, 500, 60000, 14, 60000, 14, 60000, 14], [2000, 54, 4, 500, 4000, 54, 2000, 54, 2000, 54])]

		whichFunc = num_peaks[state] - 1
		times = 2; skip = 30
		datacopy = deepcopy(data[1:training_data]); x = list(range(len(data)))
		
		popt, pcov = seriesIterativeCurveFit(func[whichFunc][0], x[1:training_data], datacopy, func[whichFunc][1])
		finalday, finalexp = totalExpected(func[whichFunc][0], popt, data)
		when97 = calcWhen(func[whichFunc][0], popt, 0.97 * finalexp, data)

		when97 = 1000 if when97 > 1000 else when97
		xlim = max(len(data)*times, when97+10)
		pred = [func[whichFunc][0](px, *popt) for px in list(range(xlim))[1:]]

		y = [func[whichFunc][0](px, *popt) for px in list(range(xlim))[1:]]
		maxcases, maxday = getMaxCases(y, data)
		newpredsave = deepcopy(y)
		newsave = deepcopy(data)
		cumpredsave = getcummulative(y)
		cumsave = getcummulative(data)


		newdf = pd.DataFrame.from_dict({state+'-pred': newpredsave, \
			state+'-dates':[(start+timedelta(days=i)).strftime("%d %b %Y") for i in list(range(1,xlim))], \
			state+'-true': newsave,\
			state+'-cum': cumpredsave}
			#state+'-predd': deadpredsave,\
			#state+'-trued': deadsave}
      , orient='index').T
		dfPlot[state+'-pred'] = pd.Series(newdf[state+'-pred'])
		dfPlot[state+'-dates'] = pd.Series(newdf[state+'-dates'])
		dfPlot[state+'-true'] = pd.Series(newdf[state+'-true'])
		dfPlot[state+'-cum'] = pd.Series(newdf[state+'-cum'])
		#dfPlot[state+'-predd'] = pd.Series(newdf[state+'-predd'])
		#dfPlot[state+'-trued'] = pd.Series(newdf[state+'-trued'])
		newcdf = pd.DataFrame.from_dict({state+'-dates':[(start+timedelta(days=i)).strftime("%d %b %Y") for i in list(range(1,xlim))], \
			state+'-cpred': cumpredsave, \
			state+'-ctrue': cumsave}
			#state+'-cpredd': cumdpredsave,\
			#state+'-ctrued': cumdsave}
      , orient='index').T
		dfcPlot[state+'-dates'] = pd.Series(newcdf[state+'-dates'])
		dfcPlot[state+'-pred'] = pd.Series(newcdf[state+'-cpred'])
		dfcPlot[state+'-true'] = pd.Series(newcdf[state+'-ctrue'])
		#dfcPlot[state+'-cpredd'] = pd.Series(newcdf[state+'-cpredd'])
		#dfcPlot[state+'-ctrued'] = pd.Series(newcdf[state+'-ctrued'])
		dates = [(start+timedelta(days=i)).strftime("%d %b %Y") for i in list(range(1,xlim))]
		fig = make_subplots(specs=[[{"secondary_y": True}]])
		fig.add_trace(go.Scatter(x=dates, y=newpredsave, name="Prediction (new cases)", marker=dict(color='#EB752C')))
		fig.add_trace(go.Bar(x=dates, y=newsave, name="True data (new cases)", marker=dict(color='#EB752C'), opacity=0.7, width=[1]*len(dates), hoverlabel=dict(bgcolor='#FF351C')))
		#fig.add_trace(go.Scatter(x=dates, y=deadpredsave, name="Prediction (deaths)", marker=dict(color='#2D58BE')), secondary_y=True)
		#fig.add_trace(go.Bar(x=dates, y=deadsave, name="True data (deaths)", marker=dict(color='#2D58BE'), opacity=0.3, width=[1]*len(dates), hoverlabel=dict(bgcolor='#1A22AB')), secondary_y=True)
		fig.update_layout(hovermode="x",
			title=state.capitalize(),
			title_x=0.5,
			title_font=dict(size=20),
			xaxis_title='Date',
			font=dict(family='Overpass', size=12, color='#212121'),
			yaxis_tickformat = ',.0f',
			xaxis = dict(dtick = 30),
			autosize=False,
		    width=700,
		    height=500,
		    legend=dict(x=0.6, y=0.9, bordercolor='Black', borderwidth=1),
		    plot_bgcolor='rgba(0,0,0,0)',
			)
		fig.update_yaxes(title_text="Number of daily new cases", gridcolor='lightgray', showline=True, linewidth=3, linecolor='orange', gridwidth=1, secondary_y=False,)
		fig.update_yaxes(tickformat = ',.0f',title_text="Number of daily deaths", secondary_y=True, gridcolor='lightgray', showline=True, linewidth=3, linecolor='blue', gridwidth=1)
		fig.write_html("plots/"+state+"_pred"+".html")
		####
		fig = make_subplots(specs=[[{"secondary_y": True}]])
		fig.add_trace(go.Scatter(x=dates, y=cumpredsave, name="Prediction (new cases)", marker=dict(color='#EB752C')))
		fig.add_trace(go.Bar(x=dates, y=cumsave, name="True data (new cases)", marker=dict(color='#EB752C'), opacity=0.7, width=[1]*len(dates), hoverlabel=dict(bgcolor='#FF351C')))
		#fig.add_trace(go.Scatter(x=dates, y=cumdpredsave, name="Prediction (deaths)", marker=dict(color='#2D58BE')), secondary_y=True)
		#fig.add_trace(go.Bar(x=dates, y=cumdsave, name="True data (deaths)", marker=dict(color='#2D58BE'), opacity=0.3, width=[1]*len(dates), hoverlabel=dict(bgcolor='#1A22AB')), secondary_y=True)
		fig.update_layout(hovermode="x",
			title=state.capitalize(),
			title_x=0.5,
			title_font=dict(size=20),
			xaxis_title='Date',
			font=dict(family='Overpass', size=12, color='#212121'),
			yaxis_tickformat = ',.0f',
			xaxis = dict(dtick = 30),
			autosize=False,
		    width=700,
		    height=500,
		    legend=dict(x=0.6, y=0.3, bordercolor='Black', borderwidth=1),
		    plot_bgcolor='rgba(0,0,0,0)',
			)
		fig.update_yaxes(title_text="Number of total cases", gridcolor='lightgray', showline=True, linewidth=3, linecolor='orange', gridwidth=1, secondary_y=False,)
		fig.update_yaxes(tickformat = ',.0f',title_text="Number of total deaths", secondary_y=True, gridcolor='lightgray', showline=True, linewidth=3, linecolor='blue', gridwidth=1)
		fig.write_html("plots/"+state+"_total"+".html")
		fig.data = []
	except Exception as e:
		print(str(e))
		raise(e)
		pass

f = open("plots/lastupdated.txt", "w")
now = datetime.now().time()
today = date.today()
f.write(now.strftime("%H:%M:%S")+" - "+today.strftime("%d %b %Y"))
f.close()
# dfPlot.to_excel('plot.xlsx')
# dfcPlot.to_excel('cplot.xlsx')

-- DELHI
[60.94670064586679, 56.931146885071726, 58.954989406815514, 57.17298385007384, 60.1588048169106]
-- HARYANA
[66.48685174822407, 67.78451118267938, 71.8757053320894, 74.88024177348024, 75.41519178734427]
-- PUNJAB
[22.763385490360985, 22.367071176697095, 21.826146090600062, 49.522589991266344, 49.156537995898915]
-- BIHAR
[80.7765049369171, 78.89676541463629, 79.60165452659027, 78.67868053045275, 78.71685084067413]
